# **ANÁLISE, EXPLORAÇÃO E TRATAMENTO DOS DADOS**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings("ignore")

https://www.kaggle.com/uciml/breast-cancer-wisconsin-data 

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Cursos_Udemy/otimizacao_hiperparametro/dados/data_cancer.csv',
                    sep=',', encoding='utf-8')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df2 = df.drop(columns=['id','Unnamed: 32'])

In [ ]:
df2.head()

In [ ]:
df2.dtypes

In [ ]:
df3 = pd.DataFrame.copy(df2)

In [ ]:
df3['diagnosis'].replace({'M':0, 'B': 1}, inplace=True)
df3.head()

In [ ]:
# Fração de tumores benignos (1) e malignos (0)

df3.diagnosis.value_counts() / len(df3.diagnosis)

# **PRÉ-PROCESSAMENTO**

## **Separação em Variáveis Previsoras e Alvo**

In [ ]:
df3.head()

In [ ]:
previsores = df3.iloc[:, 1:32].values
previsores

In [ ]:
previsores.shape

In [ ]:
alvo = df3.iloc[:, 0].values
alvo

In [ ]:
alvo.shape

**Divisão em dados de treino e teste**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(
    previsores, alvo, test_size=0.3, random_state=0)

X_treino.shape, X_teste.shape

# **SEM OTIMIZAR HIPERPARÂMETROS**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
# Desempenho validação cruzada
desempenho_cross_val = cross_val_score(estimator = DecisionTreeClassifier(random_state=0),
                                       X = X_treino, 
                                       y = y_treino,
                                       scoring = 'roc_auc',
                                       cv = 6)

print('Desempenho médio de AUC-ROC: {}'.format(round(desempenho_cross_val.mean(),4)))

# **GRID SEARCH**

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [ ]:
# Configurando a procura com o Grid search
grid_search = GridSearchCV(estimator = DecisionTreeClassifier(random_state = 0),
                              param_grid = ({'max_depth':range(1,30),'min_samples_split':range(1,5),
                                             'criterion':('gini','entropy')}),
                              scoring='roc_auc', cv = 6)

In [ ]:
# Configurando os melhores hiperparâmetros
grid_search.fit(X_treino, y_treino)

In [ ]:
# Melhores Hiperparâmetros
grid_search.best_params_

In [ ]:
print('Melhor comprimento máximo: {}'.format(grid_search.best_params_['max_depth']))
print('Melhor mínimo amostras: {}'.format(grid_search.best_params_['min_samples_split']))
print('Melhor critério: {}'.format(grid_search.best_params_['criterion']))
print('Score médio do melhor modelo: {}'.format(round(grid_search.best_score_,3)))

In [ ]:
# Relação dos dados para todos os modelos
results = pd.DataFrame(grid_search.cv_results_)
print(results.shape)
results.head(3)

In [ ]:
# Ordenando os melhores resultados
results.sort_values(by='mean_test_score', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[['param_max_depth', 'param_min_samples_split', 'param_criterion',
    'mean_test_score', 'std_test_score']].head()

In [ ]:
# ROC-AUC para dados de treino e teste
X_treino_prev = grid_search.predict_proba(X_treino)[:,1]
X_teste_prev = grid_search.predict_proba(X_teste)[:,1]

print('ROC-AUC treino: ', roc_auc_score(y_treino, X_treino_prev))
print('ROC-AUC teste: ', roc_auc_score(y_teste, X_teste_prev))

# **RANDOM SEARCH**

In [ ]:
from scipy import stats

In [ ]:
stats.randint.rvs(1, 6)

In [ ]:
stats.uniform.rvs(0, 1)

## Random Search para o classificador Árvore de Decisão.

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Determinação do espaço de hiperparâmetros
param_random = dict(
    criterion=('gini','entropy'),
    min_samples_split=stats.randint(1,8),
    max_depth=stats.randint(1, 31))

In [ ]:
random_search = RandomizedSearchCV(estimator = DecisionTreeClassifier(random_state = 0),
                              param_distributions=param_random, scoring='roc_auc',
                              cv = 6, n_iter=150)

In [ ]:
# Configurando os melhores hiperparâmetros
random_search.fit(X_treino, y_treino)

In [ ]:
# Melhores hiperparâmetros
random_search.best_params_

In [ ]:
# Resultados para todos os modelos
results = pd.DataFrame(random_search.cv_results_)
print(results.shape)
results.head(2)

In [ ]:
# Ordenando os melhores resultados
results.sort_values(by='mean_test_score', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[['param_max_depth', 'param_min_samples_split', 'param_criterion',
    'mean_test_score', 'std_test_score']].head()

In [ ]:
print('Melhor profundidade máxima: {}'.format(random_search.best_params_['max_depth']))
print('Melhor mínimo amostras: {}'.format(random_search.best_params_['min_samples_split']))
print('Melhor critério: {}'.format(random_search.best_params_['criterion']))
print('Score médio do melhor modelo: {}'.format(round(random_search.best_score_,3)))

In [ ]:
# ROC-AUC para dados de treino e teste
X_treino_prev = random_search.predict_proba(X_treino)[:,1]
X_teste_prev = random_search.predict_proba(X_teste)[:,1]

print('ROC-AUC treino: ', roc_auc_score(y_treino, X_treino_prev))
print('ROC-AUC teste: ', roc_auc_score(y_teste, X_teste_prev))

# **OTIMIZAÇÃO BAYESIANA**

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
pip install scikit-optimize

In [ ]:
# Importando o Bayes Search com cross-validation:
from skopt import BayesSearchCV

In [ ]:
# Importando espaços de busca
from skopt.space import Integer, Real, Categorical

In [ ]:
bayes_search = BayesSearchCV(estimator = DecisionTreeClassifier(random_state = 0),
                              search_spaces = {'max_depth':Integer(1,30), 
                                               'min_samples_split': Integer(2,8),
                                               'criterion': Categorical(['gini', 'entropy'])},
                              scoring='roc_auc', cv = 6)

In [ ]:
# Realizando a otimização Bayesiana
bayes_search.fit(X_treino, y_treino)

In [ ]:
#Vamos ver informações relevantes:
print('Melhor profundidade máxima: {}'.format(bayes_search.best_params_['max_depth']))
print('Melhor número mínimo de amostras: {}'.format(bayes_search.best_params_['min_samples_split']))
print('Melhor critério: {}'.format(bayes_search.best_params_['criterion']))
print('Desempenho AUC-ROC do melhor modelo: {}'.format(round(bayes_search.best_score_,3)))

In [ ]:
# ROC-AUC para dados de treino e teste
X_treino_prev = random_search.predict_proba(X_treino)[:,1]
X_teste_prev = random_search.predict_proba(X_teste)[:,1]

print('ROC-AUC treino: ', roc_auc_score(y_treino, X_treino_prev))
print('ROC-AUC teste: ', roc_auc_score(y_teste, X_teste_prev))